## Important note

I started out with a CNN for the MNIST (handwritten digits) dataset just to get a basic example working. Once I make sure that part works, I can just change the parameters to match our use case.

This notebook was an attempt to convert a Keras Sequential model (a CNN) into a Core ML model using their conversion API. However, creating the model in this manner does not result in a trainable model. The way you can see if a model is trainable is if you can see a "TrainingInput" class in the compiled model file in Xcode.

In [1]:
import keras
from keras import layers
from keras import models, Sequential
from keras.datasets import mnist
from keras.utils import to_categorical

2023-11-27 14:52:41.327104: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# This is our model. A handwritten digits predictor trained with the MNIST
# handwritten digits dataset.
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

2023-11-27 14:52:47.505257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [4]:
# compile the model
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [5]:
# The MNIST handwritten digits dataset is part of Keras. Let's get it.
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# get the data ready to pass into our model
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [59]:
# training time!
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 24s 25ms/step - loss: 0.1824 - accuracy: 0.9435
Epoch 2/5
938/938 [==============================] - 26s 28ms/step - loss: 0.0481 - accuracy: 0.9850
Epoch 3/5
938/938 [==============================] - 28s 29ms/step - loss: 0.0337 - accuracy: 0.9892
Epoch 4/5
938/938 [==============================] - 24s 26ms/step - loss: 0.0253 - accuracy: 0.9922
Epoch 5/5
938/938 [==============================] - 25s 26ms/step - loss: 0.0201 - accuracy: 0.9938


In [60]:
# let's see how good we are with the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 4ms/step - loss: 0.0242 - accuracy: 0.9921
Test accuracy: 0.9921000003814697


In [61]:
# Let's save our model to disk in Keras's h5 format
print('Saving model...')
model.save('conv_mnist.h5')
print('Done!')

Saving model...
Done!


In [79]:
from coremltools.converters import keras as keras_converter

class_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

mlmodel = keras_converter.convert(keras_model, 
                                  input_names=['image'],
                                  output_names=['digitProbabilities'],
                                  class_labels=class_labels,
                                  predicted_feature_name='digit')

mlmodel.save(mlmodel_url)

ImportError: cannot import name 'keras' from 'coremltools.converters' (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/coremltools/converters/__init__.py)

In [78]:
import coremltools as ct

class_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

mlmodel = ct.convert(model,
                     convert_to="neuralnetwork")

mlmodel.save("cnn.mlmodel")

2023-11-26 23:32:55.408781: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2023-11-26 23:32:55.457457: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2023-11-26 23:32:55.502360: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2023-11-26 23:32:55.543894: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
Translating MIL ==> NeuralNetwork Ops: 100%|████████████| 54/54 [00:00<00:00, 3655.82 ops/s]


In [69]:
id = 5
sample_input = test_images[id].reshape((1, 28, 28, 1))
result = coreml_model.predict({'conv2d_input': sample_input})
print(np.argmax(result["Identity"]), test_labels[id])

1 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [70]:
coreml_model.author = 'Jfantab'
coreml_model.license = 'BSD'
coreml_model.short_description = 'Model to predict handwritten digits'

# coreml_model.input_description['input'] = '28x28 grayscale image of digit to predict'
# coreml_model.output_description['output'] = 'Digit ASCII value (prediction)'

coreml_model.save('conv_mnist.mlpackage')

In [75]:
import coremltools
spec = coremltools.utils.load_spec('conv_mnist.mlpackage')
builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec)
builder.inspect_layers(last=3)

AttributeError: 'NoneType' object has no attribute 'layers'